# Segmentação de Microplásticos em Microscopia

Este notebook segmenta microplásticos em imagens de microscopia com fundo cinza/claro usando análise HSV otimizada.

## Desafios de Imagens de Microscopia:

**Problemas típicos:**
- Fundo cinza (não branco puro)
- Cores escuras e dessaturadas
- Baixo contraste
- Objetos pequenos
- Iluminação não uniforme

## Solução HSV Otimizada:

### 1. Detecção de Fundo por Brilho:
- **Média RGB**: Detecta fundo claro por intensidade média
- **Flexível**: Funciona com cinza, off-white, etc.

### 2. Segmentação HSV:
- **Hue (Matiz)**: Identifica cor base independente de brilho
- **Saturation (Saturação)**: Distingue cores de cinza
- **Value (Valor)**: Permite objetos escuros

### 3. Ranges Otimizados:
- **Azul**: H=100-130 (inclui azuis escuros)
- **Vermelho/Laranja**: H=0-25 ou H=170-179
- **Verde**: H=40-80
- **Escuro**: Baixo valor e saturação

In [11]:
# Importação das bibliotecas necessárias
import cv2  # OpenCV para processamento de imagens
import numpy as np  # NumPy para operações com arrays
import matplotlib.pyplot as plt  # Matplotlib para visualização
import os  # Operações do sistema operacional
from ipywidgets import interact, IntSlider, Dropdown, Checkbox  # Widgets interativos
from IPython.display import display  # Display para Jupyter

In [12]:
# Carregamento das imagens disponíveis
data_dir = '../data'  # Diretório das imagens
supported_formats = ('.jpg', '.jpeg', '.png', '.tiff', '.tif')  # Formatos suportados

image_files = []  # Lista de arquivos de imagem
if os.path.exists(data_dir):  # Verifica se diretório existe
    # Filtra apenas arquivos com extensões suportadas
    image_files = [f for f in os.listdir(data_dir) 
                   if f.lower().endswith(supported_formats)]

if not image_files:
    print("Nenhuma imagem encontrada no diretório de dados")
else:
    print(f"Encontradas {len(image_files)} imagens: {image_files}")

Encontradas 26 imagens: ['P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'PCB1.1_LK_Q5.jpg', 'P1A_T1_LD_Q5_100X.jpg', 'CNN_P1_L1_T1_M1_LG_Q8_150x.jpg', 'PCB1.1_LK_Q7.jpg', 'NIVEL 2 (BR1_1CAMP_LO_Q6_125X).jpg', 'PCB1.1_LM_.jpg', 'NIVEL 5 (P1D_T1_LI_Q11_63X).jpg', 'NIVEL 4 (P1D_T2_LJ_Q10_150X).jpg', 'P1A_T1_LC_Q6_100X.jpg', 'CNN_P1_L1_T1_M1_LH_Q3_63x.jpg', 'CNN_P1_L1_T1_M1_LI_Q2_50x.jpg', 'P1A_T1_LG_Q8_150X (1).jpg', 'PCB1.1.jpg', 'NIVEL 1 (BR3_1CAMP_LC_Q6_100X).jpg', 'NIVEL 3 (P1D_T3_LH_Q8_100X (2)).jpg', 'PCB1.1LB_Q9.jpg', 'CNN_P1_L1_T1_M1_LG_Q5_80x.jpg', 'PCB1.1_LK_Q10.jpg', 'CNN_P1_L1_T1_M1_LH_Q9_F1_150x.jpg', 'PCB1.1_LB_Q6.jpg', 'CNN_P1_L1_T1_M1_LC_Q9_100x.jpg', 'CNN_P1_L1_T1_M1_LG_Q11_100x.jpg', 'CNN_P1_L1_T1_M1_LI_Q6_125x.jpg', 'P1A_T1_LI_Q11_40X.jpg']


In [13]:
def segment_microplastics(image_path, background_threshold=160, min_saturation=20, apply_morphology=True):
    """
    Segmenta microplásticos em imagens de microscopia usando análise HSV
    
    Parâmetros:
    - image_path: caminho para a imagem
    - background_threshold: limiar para detectar fundo claro (0-255)
    - min_saturation: saturação mínima para considerar colorido (0-255)
    - apply_morphology: aplicar limpeza morfológica
    
    Retorna:
    - original: imagem original RGB
    - masks: dicionário com máscaras para cada cor
    - segmentations: dicionário com segmentações por cor
    - counts: dicionário com contagem de objetos por cor
    """
    # Carrega e converte imagem para RGB
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Converte para HSV para melhor detecção de cores
    hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    
    # Separa canais RGB para detecção de fundo
    red_channel = img_rgb[:,:,0]
    green_channel = img_rgb[:,:,1]
    blue_channel = img_rgb[:,:,2]
    
    # Detecta fundo claro usando média dos canais RGB
    # Funciona melhor que limiar individual para fundos cinza
    background_mask = ((red_channel.astype(np.float32) + 
                       green_channel.astype(np.float32) + 
                       blue_channel.astype(np.float32)) / 3) > background_threshold
    
    # Inverte para focar em objetos (não-fundo)
    object_mask = ~background_mask
    
    # Detecta azul usando HSV (melhor para azuis escuros de microscopia)
    # Azul no HSV: H=100-130 (OpenCV usa 0-179 para matiz)
    blue_hue_mask = (h >= 100) & (h <= 130)
    blue_sat_mask = s > min_saturation  # Saturação mínima para distinguir de cinza
    blue_val_mask = v > 15  # Valor mínimo baixo para azuis muito escuros
    blue_mask = object_mask & blue_hue_mask & blue_sat_mask & blue_val_mask
    
    # Detecta vermelho/laranja usando HSV
    # Vermelho: H=0-10 ou H=170-179, Laranja: H=10-25
    red_hue_mask = ((h >= 0) & (h <= 25)) | (h >= 170)
    red_sat_mask = s > min_saturation
    red_val_mask = v > 20
    red_mask = object_mask & red_hue_mask & red_sat_mask & red_val_mask
    
    # Detecta verde usando HSV
    # Verde: H=40-80
    green_hue_mask = (h >= 40) & (h <= 80)
    green_sat_mask = s > min_saturation
    green_val_mask = v > 20
    green_mask = object_mask & green_hue_mask & green_sat_mask & green_val_mask
    
    # Detecta amarelo (comum em microscopia)
    # Amarelo: H=25-40
    yellow_hue_mask = (h >= 25) & (h <= 40)
    yellow_sat_mask = s > min_saturation
    yellow_val_mask = v > 30
    yellow_mask = object_mask & yellow_hue_mask & yellow_sat_mask & yellow_val_mask
    
    # Detecta objetos escuros (pretos/cinza escuro)
    # Baixo valor E baixa saturação E não é fundo
    dark_mask = object_mask & (v < 80) & (s < min_saturation)
    
    # Remove sobreposições (prioridade: colorido > escuro)
    dark_mask = dark_mask & ~blue_mask & ~red_mask & ~green_mask & ~yellow_mask
    
    # Converte máscaras booleanas para uint8
    masks = {
        'azul': (blue_mask * 255).astype(np.uint8),
        'vermelho': (red_mask * 255).astype(np.uint8),
        'verde': (green_mask * 255).astype(np.uint8),
        'amarelo': (yellow_mask * 255).astype(np.uint8),
        'escuro': (dark_mask * 255).astype(np.uint8),
        'fundo': (background_mask * 255).astype(np.uint8)
    }
    
    if apply_morphology:
        # Kernel pequeno para objetos pequenos de microscopia
        kernel = np.ones((2,2), np.uint8)
        
        # Aplica limpeza morfológica suave
        for color in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']:
            # Opening: remove ruído pequeno
            masks[color] = cv2.morphologyEx(masks[color], cv2.MORPH_OPEN, kernel)
            # Closing: preenche buracos pequenos
            masks[color] = cv2.morphologyEx(masks[color], cv2.MORPH_CLOSE, kernel)
    
    # Cria segmentações aplicando máscaras na imagem original
    segmentations = {}
    for color in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']:
        segmentations[color] = cv2.bitwise_and(img_rgb, img_rgb, mask=masks[color])
    
    # Conta objetos usando contornos com área mínima pequena
    counts = {}
    min_area = 15  # Área mínima muito pequena para microplásticos
    
    for color in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']:
        contours, _ = cv2.findContours(masks[color], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Filtra apenas contornos muito pequenos (ruído)
        valid_contours = [c for c in contours if cv2.contourArea(c) > min_area]
        counts[color] = len(valid_contours)
    
    return img_rgb, masks, segmentations, counts

In [14]:
def interactive_microplastic_segmentation(image_name, background_threshold, min_saturation, apply_morphology):
    """
    Função interativa para segmentação de microplásticos
    
    Parâmetros:
    - image_name: nome da imagem selecionada
    - background_threshold: limiar para detectar fundo (0-255)
    - min_saturation: saturação mínima para cores (0-255)
    - apply_morphology: aplicar limpeza morfológica
    """
    if not image_files:
        print("Nenhuma imagem disponível")
        return
    
    # Constrói caminho completo da imagem
    image_path = os.path.join(data_dir, image_name)
    
    # Realiza segmentação de microplásticos
    original, masks, segmentations, counts = \
        segment_microplastics(image_path, background_threshold, min_saturation, apply_morphology)
    
    # Cria figura com 12 subplots (2 linhas x 6 colunas)
    plt.figure(figsize=(30, 10))
    
    # LINHA SUPERIOR: Original, fundo e máscaras
    
    # Subplot 1: Imagem original
    plt.subplot(2, 6, 1)
    plt.imshow(original)
    plt.title('Imagem Original\n(Microscopia)')
    plt.axis('off')
    
    # Subplot 2: Fundo detectado
    plt.subplot(2, 6, 2)
    plt.imshow(masks['fundo'], cmap='gray')
    plt.title('Fundo Detectado\n(Cinza/Claro)')
    plt.axis('off')
    
    # Subplot 3: Máscara azul
    plt.subplot(2, 6, 3)
    plt.imshow(masks['azul'], cmap='Blues')
    plt.title(f'Azul\n({counts["azul"]} objetos)')
    plt.axis('off')
    
    # Subplot 4: Máscara vermelha
    plt.subplot(2, 6, 4)
    plt.imshow(masks['vermelho'], cmap='Reds')
    plt.title(f'Vermelho/Laranja\n({counts["vermelho"]} objetos)')
    plt.axis('off')
    
    # Subplot 5: Máscara verde
    plt.subplot(2, 6, 5)
    plt.imshow(masks['verde'], cmap='Greens')
    plt.title(f'Verde\n({counts["verde"]} objetos)')
    plt.axis('off')
    
    # Subplot 6: Máscara amarela
    plt.subplot(2, 6, 6)
    plt.imshow(masks['amarelo'], cmap='YlOrBr')
    plt.title(f'Amarelo\n({counts["amarelo"]} objetos)')
    plt.axis('off')
    
    # LINHA INFERIOR: Segmentações e estatísticas
    
    # Subplot 7: Máscara escura
    plt.subplot(2, 6, 7)
    plt.imshow(masks['escuro'], cmap='gray')
    plt.title(f'Escuro/Preto\n({counts["escuro"]} objetos)')
    plt.axis('off')
    
    # Subplot 8: Segmentação azul
    plt.subplot(2, 6, 8)
    plt.imshow(segmentations['azul'])
    plt.title('Microplásticos Azuis')
    plt.axis('off')
    
    # Subplot 9: Segmentação vermelha
    plt.subplot(2, 6, 9)
    plt.imshow(segmentations['vermelho'])
    plt.title('Microplásticos Vermelhos')
    plt.axis('off')
    
    # Subplot 10: Segmentação verde
    plt.subplot(2, 6, 10)
    plt.imshow(segmentations['verde'])
    plt.title('Microplásticos Verdes')
    plt.axis('off')
    
    # Subplot 11: Segmentação amarela
    plt.subplot(2, 6, 11)
    plt.imshow(segmentations['amarelo'])
    plt.title('Microplásticos Amarelos')
    plt.axis('off')
    
    # Subplot 12: Gráfico de contagem
    plt.subplot(2, 6, 12)
    colors = ['blue', 'red', 'green', 'gold', 'black']
    labels = ['Azul', 'Vermelho', 'Verde', 'Amarelo', 'Escuro']
    values = [counts['azul'], counts['vermelho'], counts['verde'], 
              counts['amarelo'], counts['escuro']]
    
    bars = plt.bar(labels, values, color=colors, alpha=0.7)
    plt.title('Contagem de\nMicroplásticos')
    plt.ylabel('Número')
    plt.xticks(rotation=45, fontsize=8)
    
    # Adiciona valores nas barras
    for bar, value in zip(bars, values):
        if value > 0:
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                    str(value), ha='center', va='bottom', fontsize=8)
    
    # Título geral com parâmetros e total
    total_objects = sum(values)
    plt.suptitle(f'Segmentação de Microplásticos - Fundo: {background_threshold} | Saturação: {min_saturation} | Total: {total_objects}', 
                fontsize=16)
    
    # Ajusta layout
    plt.tight_layout()
    plt.show()

In [15]:
# Widget interativo para segmentação de microplásticos
if image_files:
    interact(interactive_microplastic_segmentation,
             # Dropdown para seleção da imagem
             image_name=Dropdown(options=image_files, description='Imagem:'),
             
             # Slider para limiar de fundo
             # Valores baixos = mais pixels considerados como fundo
             # Valores altos = apenas pixels muito claros são fundo
             background_threshold=IntSlider(min=120, max=200, step=10, value=160, 
                                          description='Limiar Fundo:'),
             
             # Slider para saturação mínima
             # Valores baixos = aceita cores mais acinzentadas
             # Valores altos = apenas cores muito saturadas
             min_saturation=IntSlider(min=10, max=60, step=5, value=20, 
                                    description='Saturação Mín:'),
             
             # Checkbox para limpeza morfológica
             apply_morphology=Checkbox(value=True, 
                                     description='Aplicar Limpeza Morfológica'))
else:
    print("Adicione imagens ao diretório ../data/ para usar a segmentação de microplásticos")

interactive(children=(Dropdown(description='Imagem:', options=('P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'P…

## Algoritmo Otimizado para Microscopia:

### 1. Detecção de Fundo Flexível:
```python
# Usa média RGB ao invés de limiar individual
background = (R + G + B) / 3 > limiar_fundo
```
**Vantagem**: Funciona com fundos cinza, off-white, iluminação não uniforme

### 2. Segmentação HSV Otimizada:
```python
# Azul escuro (comum em microscopia)
azul = (H=100-130) & (S>20) & (V>15) & não_fundo
```
**Vantagem**: Detecta azuis muito escuros que RGB perderia

### 3. Múltiplas Cores:
- **Azul**: H=100-130 (filamentos azuis escuros)
- **Vermelho/Laranja**: H=0-25 ou H=170-179 (partículas laranjas)
- **Verde**: H=40-80 (partículas verdes)
- **Amarelo**: H=25-40 (partículas amarelas)
- **Escuro**: Baixo V e S (partículas pretas)

### 4. Parâmetros Ajustáveis:

#### Limiar de Fundo (120-200):
- **120-140**: Remove mais pixels como fundo (fundos mais escuros)
- **160-180**: Equilibrio para fundos cinza típicos
- **190-200**: Apenas pixels muito claros são fundo

#### Saturação Mínima (10-60):
- **10-20**: Aceita cores muito dessaturadas (microscopia típica)
- **30-40**: Equilibrio entre cor e cinza
- **50-60**: Apenas cores muito puras

### 5. Área Mínima Reduzida:
- **15 pixels**: Adequado para microplásticos pequenos
- **Kernel 2x2**: Limpeza suave que preserva objetos pequenos

### Resultado Esperado:
Para sua imagem com filamento azul escuro:
- **Fundo cinza**: Será detectado e removido
- **Filamento azul**: Aparecerá na segmentação azul
- **Partículas laranjas**: Aparecerão na segmentação vermelha
- **Partículas pretas**: Aparecerão na segmentação escura